In [2]:
import json
import configparser
import requests
import time
from datetime import datetime, timedelta
from conj_gen import TLEday, pos_file, pos_tle, conj_tle_ub
import numpy as np
import pandas as pd

class MyError(Exception):
    def __init___(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args

In [33]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"
requestCmdAction       = "/basicspacedata/query" 
# requestFindSat   = "/class/satcat/OBJECT_TYPE/PAYLOAD/LAUNCH/<2010-10-01/DECAY/>2012-01-01/format/json/orderby/NORAD_CAT_ID%20asc"
requestFindSat   = "/class/satcat/OBJECT_TYPE/PAYLOAD/LAUNCH/<2010-10-01/DECAY/null-val/format/json/orderby/NORAD_CAT_ID%20asc"

config = configparser.ConfigParser()
config.read("./SpTrack-2.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")
siteCred = {'identity': configUsr, 'password': configPwd}

In [18]:
%%time
# obtain list of NORAD CAT IDs fitting parameters (launch < 2010, decay > 2012 or null)
with requests.Session() as session:

    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")

    resp = session.get(uriBase + requestCmdAction + requestFindSat)
    if resp.status_code != 200:
        print(resp)
        raise MyError(resp, "GET fail on request for satellites")

    retData = json.loads(resp.text)
    satCount = len(retData)
    satIds = []
    for e in retData:
        catId = e['NORAD_CAT_ID']
        satIds.append(catId)

    with open("satIds_2.json", 'w') as f:
        json.dump(satIds, f)
    
session.close()

CPU times: total: 15.6 ms
Wall time: 4.92 s


In [178]:
%%time
# generate conjunctions. 300 satellites take 1 hr 15 min. chose 700 km distance between satellites to be considered
sat_id = []
int_index = []
conj_time = []
conj_dist = []

with open ("satIds_2.json", 'r') as f:
    satIds = json.load(f)
total = len(satIds)

# satIds_1.json - commented means has run already
# satIds = satIds[:45]
# satIds = satIds[45:50]
# satIds = satIds[50:70]
# satIds = satIds[70:100]
# satIds = satIds[100:150]
# satIds = satIds[150:200]
# satIds = satIds[200:300]
# satIds = satIds[300:400]
# satIds = satIds[400:445]
# satIds = satIds[445:]

# missing data -- noTLE entries
# satIds = satIds[45:200] # not necessary... 

# satIds_2.json
# satIds = satIds[:5]
# satIds = satIds[5:10]
# satIds = satIds[10:300]
# satIds = satIds[300:551]
# satIds = satIds[551:553] ISSUE -- 552 TLE NOT READABLE (7736)
# satIds = satIds[553:700]
# satIds = satIds[700:970] ISSUE -- 970 TLE NOT READABLE (12783)
# satIds = satIds[971:1000]
# satIds = satIds[1000:1300]
# satIds = satIds[1300:1500]
# satIds = satIds[1500:1800]
# satIds = satIds[1800:2100]
# satIds = satIds[2100:2400]
# satIds = satIds[2400:2700]
# satIds = satIds[2700:3000]
# satIds = satIds[3000:3184] ISSUE -- 3184 TLE NOT READABLE (35946)
satIds = satIds[3185:]

with requests.Session() as session:
    
    resp = session.post(uriBase + requestLogin, data = siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    maxs = 1
    for s in satIds:
        print(s)
        requestTLE = "/class/gp_history/NORAD_CAT_ID/" + str(s) + "/EPOCH/2010-10-01--2011-12-01/orderby/PERIAPSIS%20asc/format/json"
        resp = session.get(uriBase + requestCmdAction + requestTLE)
        retData = json.loads(resp.text)
        maxs+=1
        if maxs > 10:
            print("Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...")
            time.sleep(120)
            maxs = 1
        tle = []
        if not retData:
            print("No historical TLE data for time range.")
            no_TLE = np.append(no_TLE, s)
            continue
        elif float(retData[0]['PERIAPSIS']) < 1000:
            for t in retData:
                tle = np.append(tle, [[t['TLE_LINE1']], [t['TLE_LINE2']]])
            a = conj_tle_ub(tle, 700)
            b = a['int-start'] + a['min-s'].apply(lambda x: timedelta(seconds = x))
            for i in range(a.shape[0]):
                sat_id.append(s)
            int_index.extend(a.index.values)
            conj_time.extend(b.values)
            conj_dist.extend(a['min-km'].values)
        else:
            continue

session.close()

35951
No historical TLE data for time range.
36032
36033
36036
36037
36086
No historical TLE data for time range.
36088
36095
36097
36101
Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...
36104
No historical TLE data for time range.
36106
36108
No historical TLE data for time range.
36110
36111
36112
36113
36119
36121
36122
Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...
36131
36287
36358
36395
36397
36400
36401
36402
36411
36413
Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...
36414
36415
36499
36508
36514
No historical TLE data for time range.
36516
36519
36576
No historical TLE data for time range.
36577
No historical TLE data for time range.
36578
Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...
No historical TLE data for time range.
36581
36582
36585
36588
36590
36592
36596
36598
36599
36605
Snoozing for 120 secs for rate limit reasons (max 30/min and 300/hr)...
36608
No historical TLE data

In [187]:
# run this to check index of satellite whose TLE has failed to propagate

# satIds.index('35946') # problematic
print(total) # check size of NORAD CAT ID array

3267


In [183]:
# run this to pickle data

store = pd.DataFrame({'sat-ID': sat_id, 'int-index': int_index, 'conj-time': conj_time, 'conj-dist': conj_dist})
store.to_pickle('1_3185_3267.pkl')

In [186]:
# run np.save to append new no_TLEs to the .npy file

print(len(no_TLE), no_TLE[-1])
# np.save('noTLE45_200.npy', no_TLE)

In [184]:
# run this to check data

pd.read_pickle('0_45.pkl').sort_values(by=['conj-time'])
pd.read_pickle('1_3185_3267.pkl').sort_values(by=['conj-dist'])

,sat-ID,int-index,conj-time,conj-dist
9,36119,100,2011-06-07 20:59:18,90.873911
46,36827,119,2011-07-20 19:23:11,103.517830
31,36599,119,2011-07-20 19:23:11,106.044158
47,36827,34,2011-01-09 15:28:50,128.749864
32,36599,34,2011-01-09 15:28:50,131.581460
27,36598,25,2010-12-20 07:13:58,134.363275
39,36796,83,2011-04-30 10:37:40,140.992211
28,36598,29,2010-12-29 08:14:51,171.427147
0,36036,116,2011-07-14 00:40:03,177.003077
20,36596,10,2010-11-16 09:22:35,178.152586


In [188]:
# RUN ONCE... ALREADY RUN
with open ("satIds_1.json", 'r') as f:
    check = json.load(f)
len(check)
# no_TLE = []
# no_TLE.extend(check[200:445])

517